In [ ]:
import os
from speechbrain.augment.time_domain import DropChunk, DropFreq, AddReverb, AddNoise, DropBitResolution, SpeedPerturb
import soundfile as sf
import pandas as pd
import IPython.display as ipd
from IPython.display import Audio, display, HTML
import torch

In [ ]:
os.chdir('../.')

In [ ]:
path1 = 'data/voxceleb/voxceleb1_2/id00012/21Uxsk56VDQ/00001.wav'
path2 = 'data/voxceleb/voxceleb1_2/id00012/21Uxsk56VDQ/00002.wav'

audio1, sr1 = sf.read(path1)
audio2, sr2 = sf.read(path2)

assert sr1 == sr2
sr = sr1

min_len = min(len(audio1), len(audio2))
if min_len == len(audio1):
    audio2 = audio2[: min_len]
else:
    audio1 = audio1[: min_len]


audio1 = torch.tensor(audio1).float().unsqueeze(0)
audio2 = torch.tensor(audio2).float().unsqueeze(0)
audio = torch.cat((audio1, audio2), dim=0)
audio.shape

## DropChunk

In [ ]:
chunk_dropper = DropChunk(drop_length_low=2000, drop_length_high=8000)
chunk_dropped = chunk_dropper(audio, torch.tensor([1, 1]))

## DropFreq

In [ ]:
freq_dropper = DropFreq(drop_freq_count_low=1, drop_freq_count_high=3)
freq_dropped = freq_dropper(audio)

## AddReverb

In [ ]:
reverb = AddReverb('data/voxceleb/RIRS_NOISES/reverb.csv')
reverbed = reverb(audio)

## AddNoise

In [ ]:
noisifier = AddNoise('data/voxceleb/RIRS_NOISES/noise.csv')
noisy = noisifier(audio, lengths=torch.tensor([1, 1]))

## DropBitResolution

In [ ]:
bit_dropper = DropBitResolution()
bit_dropped = bit_dropper(audio)

## SpeedPerturb

In [ ]:
speed_perturber = SpeedPerturb(orig_freq=sr, speeds=[90, 110], device='cuda')
speeded = speed_perturber(audio)

In [ ]:
# Function that returns the HTML for an audio player
def audio_player_html(audio_array, sr):
    # Create an IPython Audio object and return its HTML representation
    return Audio(audio_array, rate=sr)._repr_html_()

# --- Example audio data ---
# Define paths for different samples under different conditions
audio_data = {
    'Sample': [f"{path1[path1.index('voxceleb1_2') + len('voxceleb1_2') + 1:]}", 
               f"{path2[path2.index('voxceleb1_2') + len('voxceleb1_2') + 1:]}"
              ],
    'Clean': [audio[0], audio[1] ],
    'Noisy': [noisy[0], noisy[1]],
    'Reverberated': [reverbed[0], reverbed[1]],
    'Frequency Dropped': [freq_dropped[0], freq_dropped[1] ],
    'Chunks Dropped': [chunk_dropped[0], chunk_dropped[1]],
    'Bit Dropped': [bit_dropped[0], bit_dropped[1]],
    'Speed Pertubed': [speeded[0], speeded[1]]
}

# Build the HTML table dynamically
headers = list(audio_data.keys())
table_html = '<table border="1" style="border-collapse: collapse; text-align: center;">'

# Create header row with centered headings
table_html += '<tr>'
for header in headers:
    table_html += f'<th style="padding: 8px; text-align: center;">{header}</th>'
table_html += '</tr>'

# Determine the number of samples (assuming all lists are the same length)
n_samples = len(audio_data['Sample'])

# Create table rows for each sample
for i in range(n_samples):
    row_html = '<tr>'
    for key in headers:
        if key == 'Sample':
            row_html += f'<td style="padding: 8px;">{audio_data[key][i]}</td>'
        else:
            audio_array = audio_data[key][i]
            player_html = audio_player_html(audio_array, sr)
            row_html += f'<td style="padding: 8px;">{player_html}</td>'
    row_html += '</tr>'
    table_html += row_html

table_html += '</table>'

# Display the complete table in the Jupyter Notebook
display(HTML(table_html))